In [1]:
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
from metpy.units import units
import metpy.calc as mpcalc
from dateutil import tz

from datetime import datetime

In [69]:
'''
Define Model Parameters
'''
MODEL_DATE = '20230625'
MODEL_CYCLE = '12z'

# Define area of interest
LAT = 39.87
LON = -75.24

In [70]:
base_url = f'https://nomads.ncep.noaa.gov/dods/gefs/gefs{MODEL_DATE}/gefs_pgrb2ap5_all_{MODEL_CYCLE}'

ds = xr.open_dataset(base_url, decode_times=True)

t_max = ds.tmax2m.metpy.sel(lat=LAT, lon=360+LON, method='nearest') * units('K')

/Users/rhawthorne/miniconda3/envs/metpy/lib/python3.11/site-packages/xarray/coding/times.py:154: SerializationWarning:

Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.



In [71]:
t_max = t_max.metpy.convert_units('degF')
t_max

Magnitude,[[nan 90.05751037597656 89.70100402832031 ... 94.54564666748047 88.38687133789062 78.52999877929688] [nan 90.1369400024414 87.10729217529297 ... 95.08391571044922 85.15568542480469 77.44998168945312] [nan 90.0512466430664 89.50967407226562 ... 76.54998779296875 75.01453399658203 76.34992218017578] ... [nan 88.79633331298828 88.84505462646484 ... 87.52999114990234 83.60012817382812 76.74005126953125] [nan 89.14998626708984 88.68009948730469 ... 91.10417938232422 79.75299072265625 73.4996337890625] [nan 87.87205505371094 88.07003021240234 ... 78.8288803100586 76.14090728759766 72.45219421386719]]
Units,degree_Fahrenheit


In [72]:
df = t_max.to_dataframe('temp')
df.dropna(inplace=True)
df.reset_index(inplace=True)
df

,ens,time,lat,lon,temp
0,1.0,2023-06-25 18:00:00,40.0,285.0,90.057510
1,1.0,2023-06-26 00:00:00,40.0,285.0,89.701004
2,1.0,2023-06-26 06:00:00,40.0,285.0,83.219727
3,1.0,2023-06-26 12:00:00,40.0,285.0,74.929993
4,1.0,2023-06-26 18:00:00,40.0,285.0,91.325607
...,...,...,...,...,...
1979,31.0,2023-07-10 12:00:00,40.0,285.0,70.583420
1980,31.0,2023-07-10 18:00:00,40.0,285.0,73.209045
1981,31.0,2023-07-11 00:00:00,40.0,285.0,78.828880
1982,31.0,2023-07-11 06:00:00,40.0,285.0,76.140907


In [73]:
df_mean = df.groupby(['time'], as_index=False).mean()
df_median = df.groupby(['time'], as_index=False).median()
df_max = df.groupby(['time'], as_index=False).max()
df_p25 = df.groupby(['time'], as_index=False).quantile(.25)
df_p75 = df.groupby(['time'], as_index=False).quantile(.75)

In [74]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['time'], y=df_max['temp'], name='Max'))
fig.add_trace(go.Scatter(x=df['time'], y=df_p75['temp'], name='p75'))
fig.add_trace(go.Scatter(x=df['time'], y=df_mean['temp'], name='Mean'))
fig.add_trace(go.Scatter(x=df['time'], y=df_median['temp'], name='p50'))
fig.add_trace(go.Scatter(x=df['time'], y=df_p25['temp'], name='p25'))

# Set layout options
fig.update_layout(title=f'Maximum Temperature at {LAT}, {LON}<br>NOAA GFS Ensemble Model',
                  xaxis_title='Time',
                  yaxis_title='Temperature (degF)')

# Display the graph
fig.show()

In [75]:
fig.write_image("timeseries-gefs_maxt.png", width=1280, height=720, scale=1)